Red más pequeña pero con GAES, se compara con el V1

In [ ]:
import os

import random
import gym
import pylab
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt

#tf.config.experimental_run_functions_eagerly(True)
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import copy

EPISODES = 100000 # Number of times the enviroment is ran
LR = 0.02 # Learning rate 
NUM_ACTIONS = 2 # Number of possible action in the environment
NUM_STATES = 4 # Number of possible states in the exvironment
EPOCHS = 10 # Epochs to train the network (recommended between 3 and 30)
BATCH_SIZE = 64 # Batch size for the neural nets
BUFFER_SIZE = 2048 # Buffer of experiences
SHUFFLE = True # Whether to shuffle data or not while training
OPTIMIZER = Adam # Optimizer for both actor and critic
GAMMA = 0.99 # Used for the estimated reward
NORMALIZE = True # Whether to normalize GAE or not



# Create the actor used to select the action given an state
class Actor_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        X = Dense(24, activation="relu", kernel_initializer='he_uniform')(X_input)
        
        # Softmax as there are different probabilities depending on the action
        output = Dense(NUM_ACTIONS, activation="softmax")(X)
        
        # Compile the model with the custom loss
        self.model = Model(inputs = X_input, outputs = output)
        self.model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER(lr=LR))

# Create the critic which will criticise how the actor is performing    
class Critic_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        X = Dense(24, activation="relu", kernel_initializer='he_uniform')(X_input)
        
        # Linear output to know how good the action is
        value = Dense(1)(X)
        
        # Compile it with mse loss and gradient descent
        self.model = Model(inputs=X_input, outputs = value)
        self.model.compile(loss='mse', optimizer=OPTIMIZER(lr=LR))

# Combine both Actor and Critic to create the agent
class PPOAgent:
    def __init__(self, env_name):
        # Environment parameters
        self.env_name = env_name       
        self.env = gym.make(env_name)
        self.episode = 0 # used to track current number episoded since start
        self.max_average = 0 # record max average reached
        
        # Used to plot a grapgh of the train process
        self.scores_, self.average_ = [], []

        # Create Actor-Critic network models
        self.Actor = Actor_Model()
        self.Critic = Critic_Model()
    
        # Names for the models
        self.Actor_name = f"{self.env_name}_PPO_Actor.h5"
        self.Critic_name = f"{self.env_name}_PPO_Critic.h5"

    # Get the action given the current state    
    def act(self, state):
        # Use the network to predict the next action to take, using the model
        prediction = self.Actor.model.predict(state)[0]
        
        # Probability based to choose the action
        action = np.random.choice(NUM_ACTIONS, p=prediction)
        action_onehot = np.zeros([NUM_ACTIONS])
        action_onehot[action] = 1
        return action, action_onehot, prediction

    # Generalized Advantage Estimation implemented in the original paper
    def get_gaes(self, rewards, dones, values, next_values):
        # Dones are used to track when is the final step of an episode, so next values are no applied
        deltas = [r + GAMMA * (1 - d) * nv - v for r, d, nv, v in zip(rewards, dones, next_values, values)]
        
        # Convert list to array as .mean() and .std() are used later
        deltas = np.stack(deltas)
        gaes = copy.deepcopy(deltas)
        
        for t in reversed(range(len(deltas) - 1)):
            gaes[t] = gaes[t] + (1 - dones[t]) * GAMMA * LAMBDA * gaes[t + 1]

        target = gaes + values
        if NORMALIZE:
            gaes = (gaes - gaes.mean()) / (gaes.std() + 1e-8)
        return np.vstack(gaes), np.vstack(target)

    def replay(self, states, actions, rewards, predictions, dones, next_states):
        # Reshape memory to appropriate shape for training
        states = np.vstack(states)
        next_states = np.vstack(next_states)
        actions = np.vstack(actions)
        predictions = np.vstack(predictions)

        # Get Critic network predictions for state and next state
        values = self.Critic.model.predict(states)
        next_values = self.Critic.model.predict(next_states)

        # Get the advantage
        advantages, target = self.get_gaes(rewards, dones, np.squeeze(values), np.squeeze(next_values))

        # Stack info to unpack it in the custom loss
        y_true = actions 
        advantages = np.reshape(advantages, (2048))

        start_train = time.time()
        # Training Actor and Critic networks
        a_loss = self.Actor.model.fit(states, y_true, sample_weight=advantages, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)
        c_loss = self.Critic.model.fit(states, target, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)
        #print('Time: ', time.time()-start_train)
 
    def load(self):
        self.Actor.Actor.load_weights(self.Actor_name)
        self.Critic.Critic.load_weights(self.Critic_name)

    def save(self):
        self.Actor.model.save_weights(self.Actor_name)
        self.Critic.model.save_weights(self.Critic_name)

    def run_batch(self): # train every self.Training_batch episodes
        global LR
        state = self.env.reset()
        state = np.reshape(state, [1, NUM_STATES])
        done, score = False, 0
        finished = False
        while finished == False:
            # Instantiate or reset games memory
            states, next_states, actions, rewards, predictions, dones = [], [], [], [], [], []
            for t in range(BUFFER_SIZE):
                #self.env.render()
                # Actor picks an action
                action, action_onehot, prediction = self.act(state)
                # Retrieve new state, reward, and whether the state is terminal
                next_state, reward, done, _ = self.env.step(action)
                # Memorize (state, action, reward) for training
                states.append(state)
                next_states.append(np.reshape(next_state, [1, NUM_STATES]))
                actions.append(action_onehot)
                rewards.append(reward)
                dones.append(done)
                predictions.append(prediction)
                # Update current state
                state = np.reshape(next_state, [1, NUM_STATES])
                score += reward
                if done:
                    self.episode += 1
                    self.scores_.append(score)
                    #average, SAVING = self.PlotModel(score, self.episode)
                    #print("episode: {}/{}, score: {}, average: {:.2f} {}".format(self.episode, EPISODES, score, average, SAVING))
                    if self.episode >= 100:
                        average = sum(self.scores_[-100:])/100
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, average))
                        if average > self.max_average:
                            self.max_average = average
                            if self.max_average > 150:
                                self.save()
                            #LR *= 0.99
                            #K.set_value(self.Actor.model.optimizer.learning_rate, LR)
                            #K.set_value(self.Critic.model.optimizer.learning_rate, LR)
                            
                        if average > 200:
                            plt.plot(self.scores_)
                            plt.xlabel("Episode")
                            plt.ylabel("Score")
                            finished = True
                            break

                    else:
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, sum(self.scores_)/len(self.scores_)))
                    
                    
                    state, done, score = self.env.reset(), False, 0
                    state = np.reshape(state, [1, NUM_STATES])
                    
            self.replay(states, actions, rewards, predictions, dones, next_states)
            if self.episode >= EPISODES:
                break
        self.env.close()  
                    
if __name__ == "__main__":
    start = time.time()
    env_name = 'CartPole-v0'
    agent = PPOAgent(env_name)
    #agent.run() # train as PPO, train every epesode
    agent.run_batch() # train as PPO, train every batch, trains better
    #agent.run_multiprocesses(num_worker = 8)  # train PPO multiprocessed (fastest)
    #agent.test()
    print((time.time() - start)/60)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Episode:     1		score:    9.00		average:    9.00
Episode:     2		score:   15.00		average:   12.00
Episode:     3		score:   11.00		average:   11.67
Episode:     4		score:   14.00		average:   12.25
Episode:     5		score:   13.00		average:   12.40
Episode:     6		score:   17.00		average:   13.17
Episode:     7		score:   10.00		average:   12.71
Episode:     8		score:   37.00		average:   15.75
Episode:     9		score:   14.00		average:   15.56
Episode:    10		score:   29.00		average:   16.90
Episode:    11		score:   11.00		average:   16.36
Episode:    12		score:   16.00		average:   16.33
Episode:    13		score:   19.00		average:   16.54
Episode:    14		score:   14.00		average:   16.36
Episode:    15		score:   20.00		average:   16.60
Episode:    16		score:   14.00		average:   16.44
Episode:    17		score:   23.00		average:   16.82
Episode:    18		score:   12.00		average:   16.56
Episode:    19		score:   11.00		avera

Episode:   161		score:  153.00		average:   71.65
Episode:   162		score:  166.00		average:   73.11
Episode:   163		score:  148.00		average:   74.47
Episode:   164		score:  134.00		average:   75.63
Episode:   165		score:   65.00		average:   75.95
Episode:   166		score:   55.00		average:   76.27
Episode:   167		score:   58.00		average:   76.75
Episode:   168		score:  200.00		average:   78.58
Episode:   169		score:   56.00		average:   79.01
Episode:   170		score:   71.00		average:   79.59
Episode:   171		score:  200.00		average:   81.44
Episode:   172		score:  200.00		average:   83.20
Episode:   173		score:  200.00		average:   85.10
Episode:   174		score:   80.00		average:   85.68
Episode:   175		score:  117.00		average:   85.95
Episode:   176		score:   77.00		average:   86.61
Episode:   177		score:  163.00		average:   88.02
Episode:   178		score:  132.00		average:   89.14
Episode:   179		score:   71.00		average:   89.62
Episode:   180		score:  139.00		average:   90.86
Episode:   181		scor

Episode:   330		score:  200.00		average:  126.02
Episode:   331		score:   63.00		average:  125.32
Episode:   332		score:  200.00		average:  126.69
Episode:   333		score:   73.00		average:  126.31
Episode:   334		score:  138.00		average:  127.16
Episode:   335		score:   67.00		average:  125.91
Episode:   336		score:   89.00		average:  126.21
Episode:   337		score:  200.00		average:  127.46
Episode:   338		score:  154.00		average:  127.00
Episode:   339		score:  162.00		average:  127.74
Episode:   340		score:  121.00		average:  127.41
Episode:   341		score:   57.00		average:  126.64
Episode:   342		score:   53.00		average:  126.56
Episode:   343		score:   55.00		average:  125.56
Episode:   344		score:   56.00		average:  124.12
Episode:   345		score:  158.00		average:  124.39
Episode:   346		score:  157.00		average:  125.39
Episode:   347		score:   53.00		average:  125.37
Episode:   348		score:  127.00		average:  125.93
Episode:   349		score:   45.00		average:  125.17
Episode:   350		scor

Episode:   499		score:  200.00		average:  109.62
Episode:   500		score:  196.00		average:  110.12
Episode:   501		score:  105.00		average:  109.17
Episode:   502		score:   39.00		average:  108.93
Episode:   503		score:  200.00		average:  108.93
Episode:   504		score:   55.00		average:  107.48
Episode:   505		score:  142.00		average:  106.90
Episode:   506		score:  200.00		average:  107.33
Episode:   507		score:  200.00		average:  108.74
Episode:   508		score:   85.00		average:  109.16
Episode:   509		score:  106.00		average:  109.00
Episode:   510		score:   55.00		average:  109.07
Episode:   511		score:   81.00		average:  109.45
Episode:   512		score:   55.00		average:  109.24
Episode:   513		score:   45.00		average:  109.23
Episode:   514		score:   65.00		average:  108.81
Episode:   515		score:   64.00		average:  108.69
Episode:   516		score:   79.00		average:  108.39
Episode:   517		score:   59.00		average:  108.34
Episode:   518		score:  200.00		average:  109.73
Episode:   519		scor

Episode:   668		score:  200.00		average:  105.21
Episode:   669		score:   53.00		average:  105.08
Episode:   670		score:   59.00		average:  105.08
Episode:   671		score:  128.00		average:  105.32
Episode:   672		score:   51.00		average:  105.18
Episode:   673		score:   46.00		average:  105.00
Episode:   674		score:  200.00		average:  105.00
Episode:   675		score:   44.00		average:  104.68
Episode:   676		score:   49.00		average:  104.65
Episode:   677		score:   71.00		average:  103.36
Episode:   678		score:   52.00		average:  103.28
Episode:   679		score:   74.00		average:  103.44
Episode:   680		score:   89.00		average:  103.92
Episode:   681		score:   61.00		average:  103.95
Episode:   682		score:   81.00		average:  102.96
Episode:   683		score:  161.00		average:  102.57
Episode:   684		score:   97.00		average:  102.72
Episode:   685		score:  138.00		average:  103.29
Episode:   686		score:  160.00		average:  103.98
Episode:   687		score:   78.00		average:  103.47
Episode:   688		scor

...

Episode: 59337		score:   10.00		average:    9.36
Episode: 59338		score:   10.00		average:    9.37
Episode: 59339		score:    9.00		average:    9.38
Episode: 59340		score:    9.00		average:    9.37
Episode: 59341		score:    9.00		average:    9.36
Episode: 59342		score:   10.00		average:    9.37
Episode: 59343		score:   10.00		average:    9.38
Episode: 59344		score:    9.00		average:    9.39
Episode: 59345		score:   10.00		average:    9.41
Episode: 59346		score:    9.00		average:    9.40
Episode: 59347		score:    8.00		average:    9.38
Episode: 59348		score:    9.00		average:    9.37
Episode: 59349		score:    9.00		average:    9.37
Episode: 59350		score:   10.00		average:    9.38
Episode: 59351		score:   10.00		average:    9.38
Episode: 59352		score:    8.00		average:    9.38
Episode: 59353		score:    9.00		average:    9.38
Episode: 59354		score:   10.00		average:    9.39
Episode: 59355		score:    9.00		average:    9.39
Episode: 59356		score:   10.00		average:    9.39
Episode: 59357		scor

Episode: 59516		score:    9.00		average:    9.42
Episode: 59517		score:    9.00		average:    9.42
Episode: 59518		score:    9.00		average:    9.42
Episode: 59519		score:   10.00		average:    9.43
Episode: 59520		score:    9.00		average:    9.42
Episode: 59521		score:   10.00		average:    9.42
Episode: 59522		score:   11.00		average:    9.43
Episode: 59523		score:   10.00		average:    9.44
Episode: 59524		score:   10.00		average:    9.44
Episode: 59525		score:    9.00		average:    9.43
Episode: 59526		score:    9.00		average:    9.43
Episode: 59527		score:    9.00		average:    9.41
Episode: 59528		score:   10.00		average:    9.42
Episode: 59529		score:   10.00		average:    9.43
Episode: 59530		score:   10.00		average:    9.44
Episode: 59531		score:   10.00		average:    9.44
Episode: 59532		score:   10.00		average:    9.45
Episode: 59533		score:    9.00		average:    9.46
Episode: 59534		score:    9.00		average:    9.45
Episode: 59535		score:    9.00		average:    9.45
Episode: 59536		scor

Episode: 59685		score:    9.00		average:    9.31
Episode: 59686		score:    9.00		average:    9.31
Episode: 59687		score:    9.00		average:    9.30
Episode: 59688		score:    9.00		average:    9.31
Episode: 59689		score:   10.00		average:    9.31
Episode: 59690		score:    9.00		average:    9.31
Episode: 59691		score:    9.00		average:    9.30
Episode: 59692		score:   10.00		average:    9.30
Episode: 59693		score:   10.00		average:    9.30
Episode: 59694		score:   10.00		average:    9.31
Episode: 59695		score:    8.00		average:    9.30
Episode: 59696		score:   10.00		average:    9.32
Episode: 59697		score:   10.00		average:    9.34
Episode: 59698		score:   10.00		average:    9.34
Episode: 59699		score:    9.00		average:    9.34
Episode: 59700		score:   10.00		average:    9.35
Episode: 59701		score:   10.00		average:    9.35
Episode: 59702		score:    9.00		average:    9.35
Episode: 59703		score:    8.00		average:    9.34
Episode: 59704		score:   10.00		average:    9.34
Episode: 59705		scor

Episode: 59859		score:    9.00		average:    9.43
Episode: 59860		score:   10.00		average:    9.43
Episode: 59861		score:   10.00		average:    9.44
Episode: 59862		score:    9.00		average:    9.43
Episode: 59863		score:   10.00		average:    9.43
Episode: 59864		score:   10.00		average:    9.44
Episode: 59865		score:    9.00		average:    9.43
Episode: 59866		score:    8.00		average:    9.41
Episode: 59867		score:   10.00		average:    9.41
Episode: 59868		score:   10.00		average:    9.42
Episode: 59869		score:   10.00		average:    9.42
Episode: 59870		score:    8.00		average:    9.40
Episode: 59871		score:    9.00		average:    9.41
Episode: 59872		score:   10.00		average:    9.42
Episode: 59873		score:    9.00		average:    9.41
Episode: 59874		score:    9.00		average:    9.40
Episode: 59875		score:   11.00		average:    9.42
Episode: 59876		score:    9.00		average:    9.40
Episode: 59877		score:    9.00		average:    9.39
Episode: 59878		score:   10.00		average:    9.40
Episode: 59879		scor

Episode: 60035		score:    8.00		average:    9.35
Episode: 60036		score:   10.00		average:    9.35
Episode: 60037		score:    8.00		average:    9.33
Episode: 60038		score:    9.00		average:    9.32
Episode: 60039		score:    8.00		average:    9.30
Episode: 60040		score:    9.00		average:    9.29
Episode: 60041		score:    9.00		average:    9.28
Episode: 60042		score:    9.00		average:    9.26
Episode: 60043		score:   10.00		average:    9.26
Episode: 60044		score:   11.00		average:    9.28
Episode: 60045		score:   10.00		average:    9.30
Episode: 60046		score:    9.00		average:    9.31
Episode: 60047		score:    8.00		average:    9.30
Episode: 60048		score:    9.00		average:    9.30
Episode: 60049		score:    9.00		average:    9.30
Episode: 60050		score:    9.00		average:    9.29
Episode: 60051		score:    9.00		average:    9.29
Episode: 60052		score:   10.00		average:    9.30
Episode: 60053		score:   10.00		average:    9.29
Episode: 60054		score:    9.00		average:    9.30
Episode: 60055		scor